In [ ]:
import json
import numpy as np
import faiss
# from upstage.embeddings import UpstageEmbedding  # Upstage Embedding 모델 사용

# Upstage Embedding 모델 로드 (업스테이지의 임베딩 사용)
embedding_model = UpstageEmbedding(model_name='upstage/your-korean-embedding-model')  # 모델 이름을 사용자의 모델로 수정

# JSONL 파일 읽기
data = []
with open('documents.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Helper function to split text into chunks of 300 words with 150-word overlap
def split_into_chunks(text, chunk_size=300, overlap=150):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
        if len(words[i:i + chunk_size]) < chunk_size:
            break
    return chunks

# 벡터 정규화 함수 (L2 노름으로 정규화하여 코사인 유사도 사용)
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# 각 문서에 대해 chunk를 나누고 임베딩 생성
chunks = []
chunk_doc_mapping = []
chunk_ids = []
chunk_embeddings = []
chunk_id = 0

for doc in data:
    content = doc["content"]
    doc_id = doc["docid"]
    
    # Chunk 문서 분할 (chunk 크기: 300, overlap: 150)
    content_chunks = split_into_chunks(content)
    
    # 각 chunk에 대해 임베딩을 생성
    for chunk in content_chunks:
        embedding = embedding_model.encode(chunk)  # Upstage Embedding으로 임베딩 생성
        chunk_embeddings.append(embedding)
        chunks.append(chunk)
        chunk_ids.append(f"{doc_id}_chunk_{chunk_id}")
        chunk_doc_mapping.append({
            "doc_id": doc_id,
            "chunk_id": f"{doc_id}_chunk_{chunk_id}",
            "content": chunk
        })
        chunk_id += 1

# numpy 배열로 변환 (FAISS에서 사용하기 위해)
chunk_embeddings = np.array(chunk_embeddings)

# 임베딩 벡터 정규화 (코사인 유사도 사용을 위해)
normalized_embeddings = normalize(chunk_embeddings)

# FAISS 코사인 유사도 인덱스 생성 (정규화된 임베딩 사용)
embedding_dim = normalized_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# 정규화된 임베딩 벡터를 인덱스에 추가
index.add(normalized_embeddings)

# FAISS 인덱스를 파일로 저장
faiss.write_index(index, "knn_index_cosine.faiss")

# Chunk ID와 문서 매핑 정보 저장
with open("chunk_mappings.json", "w") as f:
    json.dump(chunk_doc_mapping, f)

# Chunk ID들도 따로 저장 (나중에 검색할 때 사용)
with open("chunk_ids.json", "w") as f:
    json.dump(chunk_ids, f)

# ========================================================
# 검색 시 정규화된 쿼리 사용
# ========================================================

# FAISS 인덱스와 chunk-mapping 파일을 다시 로드
index = faiss.read_index("knn_index_cosine.faiss")

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 임베딩 생성 (Upstage Embedding으로 쿼리 임베딩)
query = "에너지 균형 유지의 중요성"
query_embedding = embedding_model.encode(query).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

# KNN에서 가장 유사한 5개의 chunk 검색
k = 5
distances, indices = index.search(normalized_query, k)

# 검색된 chunk 정보 확인 및 원본 문서 내용과 연결
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {chunk_info['doc_id']}")
    print(f"Chunk 내용: {chunk_info['content']}")
    print("-" * 50)

# ========================================================
# 이후 ColBERT나 Cross-Encoder로 해당 chunk 내용을 넘길 수 있음
# ========================================================


In [6]:
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)


import numpy as np
from openai import OpenAI

 
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)
document_result = client.embeddings.create(
    model = "solar-embedding-1-large-passage",
    input = "SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling. DUS is simple yet effective in scaling up high performance LLMs from small ones. "
).data[0].embedding

In [12]:
import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용


from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(OPENAI_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key
# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

# JSONL 파일 읽기
data = []
with open('/ir/data/documents.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Helper function to split text into chunks of 300 words with 150-word overlap
def split_into_chunks(text, chunk_size=300, overlap=150):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
        if len(words[i:i + chunk_size]) < chunk_size:
            break
    return chunks

# 벡터 정규화 함수 (L2 노름으로 정규화하여 코사인 유사도 사용)
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# 각 문서에 대해 chunk를 나누고 Upstage API를 통해 임베딩 생성
chunks = []
chunk_doc_mapping = []
chunk_ids = []
chunk_embeddings = []
chunk_id = 0

for doc in data:
    content = doc["content"]
    doc_id = doc["docid"]
    
    # Chunk 문서 분할 (chunk 크기: 300, overlap: 150)
    content_chunks = split_into_chunks(content)
    
    # 각 chunk에 대해 Upstage API로 임베딩 생성
    for chunk in content_chunks:
        document_result = client.embeddings.create(
            model="solar-embedding-1-large-passage",
            input=chunk
        ).data[0].embedding
        
        embedding = np.array(document_result)  # 임베딩을 numpy 배열로 변환
        chunk_embeddings.append(embedding)
        chunks.append(chunk)
        chunk_ids.append(f"{doc_id}_chunk_{chunk_id}")
        chunk_doc_mapping.append({
            "doc_id": doc_id,
            "chunk_id": f"{doc_id}_chunk_{chunk_id}",
            "content": chunk
        })
        chunk_id += 1

# numpy 배열로 변환 (FAISS에서 사용하기 위해)
chunk_embeddings = np.array(chunk_embeddings)

# 임베딩 벡터 정규화 (코사인 유사도 사용을 위해)
normalized_embeddings = normalize(chunk_embeddings)

# FAISS 코사인 유사도 인덱스 생성 (정규화된 임베딩 사용)
embedding_dim = normalized_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# 정규화된 임베딩 벡터를 인덱스에 추가
index.add(normalized_embeddings)

# FAISS 인덱스를 파일로 저장
faiss.write_index(index, "knn_index_cosine.faiss")

# Chunk ID와 문서 매핑 정보 저장
with open("chunk_mappings.json", "w") as f:
    json.dump(chunk_doc_mapping, f)

# Chunk ID들도 따로 저장 (나중에 검색할 때 사용)
with open("chunk_ids.json", "w") as f:
    json.dump(chunk_ids, f)

In [8]:
import faiss
print(faiss.__version__)

1.7.2


In [9]:
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

In [17]:
import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용

from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

In [18]:
# FAISS 인덱스와 chunk-mapping 파일을 다시 로드
# res = faiss.StandardGpuResources()

index = faiss.read_index("knn_index_cosine.faiss")

# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
query = "빗방울이 점점 커지게 되는 요인은?"
query_result = client.embeddings.create(
    model="solar-embedding-1-large-passage",
    input=query
).data[0].embedding

query_embedding = np.array(query_result).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

# KNN에서 가장 유사한 5개의 chunk 검색
k = 50
distances, indices = index.search(normalized_query, k)

# 검색된 chunk 정보 확인 및 원본 문서 내용과 연결
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {chunk_info['doc_id']}")
    print(f"Chunk 내용: {chunk_info['content']}")
    print("-" * 50)

검색된 Chunk ID: a35c09b4-e702-471c-9328-87108f9a1590_chunk_1373
연관된 문서 ID: a35c09b4-e702-471c-9328-87108f9a1590
Chunk 내용: 비가 오는 주요 원인은 태양이 지구를 가열하는 것입니다. 태양은 지구에 열을 전달하고, 이 열은 대기 중의 수증기를 증발시킵니다. 이 증발된 수증기는 대기 중으로 상승하면서 차가운 공기와 만나게 되고, 이로 인해 수증기가 응결되어 구름이 형성됩니다. 구름은 계속해서 성장하다가 충분한 양의 수증기가 모이면 비로 내리게 됩니다. 따라서 비는 태양의 열과 수증기의 상호작용으로 인해 발생하는 현상입니다.
--------------------------------------------------
검색된 Chunk ID: a90acfad-77f9-4b31-9aee-12253f9b810e_chunk_2588
연관된 문서 ID: a90acfad-77f9-4b31-9aee-12253f9b810e
Chunk 내용: 무지개는 자연 현상으로, 빗방울 속의 빛이 굴절과 반사를 통해 생기게 됩니다. 빗방울은 투명한 물질로 이루어져 있으며, 빛은 빗방울에 닿으면 굴절됩니다. 이 굴절된 빛은 빗방울 내부에서 반사되어 다시 외부로 나오게 됩니다. 이렇게 반사된 빛은 다시 한 번 굴절되며, 다양한 색상으로 분리됩니다. 이렇게 분리된 색상들이 빗방울에서 나오면서 무지개가 형성되는 것입니다. 무지개는 일정한 각도로 형성되며, 태양이 빗방울에 닿는 각도와 빗방울의 크기에 따라 다양한 형태와 색상을 보여줍니다. 따라서, 무지개는 빗방울 속의 빛이 굴절과 반사를 통해 생기는 자연의 아름다운 현상입니다.
--------------------------------------------------
검색된 Chunk ID: 0bb52aae-bcc6-498c-aa5e-7be563846857_chunk_672
연관된 문서 ID: 0bb52aae-bcc6-498c-aa5e-7be563846857
Chu

In [19]:
import numpy as np

import torch
model_path = 'Dongjin-kr/ko-reranker'

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()
    
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

In [34]:
query = "복숭아 키우는 노하우좀?"
query_result = client.embeddings.create(
    model="solar-embedding-1-large-passage",
    input=query
).data[0].embedding

query_embedding = np.array(query_result).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

# FAISS로 50개의 유사한 chunk 검색
k = 50
distances, indices = index.search(normalized_query, k)

retrieved_chunks = []
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    content = chunk_info['content']
    doc_id = chunk_info['doc_id']
    retrieved_chunks.append((query, content))
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {doc_id}")
    print(f"Chunk 내용: {content}")
    print("-" * 50)


검색된 Chunk ID: bf023a9d-47f3-4d77-8bf1-5a854b5403a7_chunk_1069
연관된 문서 ID: bf023a9d-47f3-4d77-8bf1-5a854b5403a7
Chunk 내용: 복숭아나무는 다른 복숭아보다 병에 대한 저항력이 더 큰 복숭아를 생산합니다. 이러한 특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다. 복숭아나무는 꽃이 핀 후에 수술을 통해 수술한 꽃을 다른 복숭아나무에 이식하여 교배를 진행합니다. 이렇게 교배된 복숭아는 특정 병에 대한 저항력이 더 큰 특성을 가지게 됩니다. 이 방법을 통해 복숭아나무는 더욱 강건한 복숭아를 생산할 수 있습니다. 이러한 복제 방법은 복숭아 생산자들에게 매우 중요한 기술이며, 복숭아의 품질과 수량을 향상시키는 데 큰 도움이 됩니다.
--------------------------------------------------
검색된 Chunk ID: d40a417d-48e4-43cc-8b43-47607ef925d1_chunk_2964
연관된 문서 ID: d40a417d-48e4-43cc-8b43-47607ef925d1
Chunk 내용: 사과나무에 사과가 자라기 시작할 것이라는 조짐은 나무에 꽃이 자랄 때입니다. 사과나무는 봄에 꽃을 피우는데, 이 꽃은 사과의 열매가 될 가능성을 알려줍니다. 꽃이 피면 사과나무는 열심히 꽃을 수분과 영양분으로 변환하여 사과를 키워냅니다. 사과는 여름과 가을에 점점 커지고 익어가며, 사과나무의 열매로 자랍니다. 사과나무는 사과를 통해 자신의 생명력을 전달하고 번식합니다. 따라서 사과나무에 사과가 자라기 시작할 때, 사람들은 사과의 열매가 풍성하게 열리기를 기대하며 기쁨을 느낍니다.
--------------------------------------------------
검색된 Chunk ID: 6335780f-292d-49da-a79c-3eee1d51a903_chunk_2802
연관된 문서 ID: 6335780f-292

{"docid": "82b095fd-2fb6-48ae-8476-2f457f8b6650", "src": "ko_ai2_arc__ARC_Challenge__train", "content": "인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다."}


In [35]:

# 모델에 입력하고 유사도 계산
with torch.no_grad():
    inputs = tokenizer(retrieved_chunks, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()

# 상위 5개의 문서 선택 (내림차순 정렬)
top_5_indices = torch.argsort(scores, descending=True)[:5]

# 상위 5개의 문서 출력
print("상위 5개의 문서:")
for i in top_5_indices:
    doc_index = retrieved_chunks[i][0]
    doc_text = retrieved_chunks[i][1]
    print(f"문서 인덱스: {doc_index}")
    print(f"문서 내용: {doc_text}")
    print(f"유사도 점수: {scores[i].item()}")
    print("-" * 50)

상위 5개의 문서:
문서 인덱스: 복숭아 키우는 노하우좀?
문서 내용: 복숭아나무는 다른 복숭아보다 병에 대한 저항력이 더 큰 복숭아를 생산합니다. 이러한 특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다. 복숭아나무는 꽃이 핀 후에 수술을 통해 수술한 꽃을 다른 복숭아나무에 이식하여 교배를 진행합니다. 이렇게 교배된 복숭아는 특정 병에 대한 저항력이 더 큰 특성을 가지게 됩니다. 이 방법을 통해 복숭아나무는 더욱 강건한 복숭아를 생산할 수 있습니다. 이러한 복제 방법은 복숭아 생산자들에게 매우 중요한 기술이며, 복숭아의 품질과 수량을 향상시키는 데 큰 도움이 됩니다.
유사도 점수: -3.804539680480957
--------------------------------------------------
문서 인덱스: 복숭아 키우는 노하우좀?
문서 내용: 씨앗이 완전히 어두운 흙 속에 묻힌 경우, 싹은 중력과 반대 방향으로 자라면서 흙 표면을 찾아갑니다. 이는 싹이 빛을 향해 자라기 위해 노력하는 자연의 원리입니다. 씨앗은 흙 속에서 영양분을 흡수하고, 물을 흡수하여 팽창합니다. 그리고 싹은 중력에 반항하여 흙 위로 자라나게 됩니다. 이러한 과정을 통해 씨앗은 흙 속에서 빛과 공기를 찾아나가며 생명력을 발휘합니다. 따라서, 씨앗의 싹은 중력과 반대 방향으로 자라면서 흙 표면을 찾아가는 것입니다.
유사도 점수: -5.589482307434082
--------------------------------------------------
문서 인덱스: 복숭아 키우는 노하우좀?
문서 내용: 화분에 심은 콩 씨앗은 한 개의 화분은 옷장에 두고 다른 하나는 햇빛이 잘 드는 창가에 두었습니다. 매일, 각 화분에 15밀리리터의 물을 주었습니다. 이 두 화분에서 다른 요소는 빛의 양입니다. 옷장에 둔 화분은 햇빛이 잘 들지 않아 적은 양의 빛을 받게 되고, 창가에 둔 화분은 햇빛이 잘 들어와 많은 양의 빛을 받게 됩니다. 이러한 차이로 인

In [20]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('bongsoo/kpf-cross-encoder-v1')

/opt/conda/envs/ir/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:

# 50개의 문서 쌍을 Cross-Encoder로 유사도를 평가하여 상위 5개 문서 선택
scores = model.predict(retrieved_chunks)
top_5_indices = np.argsort(scores)[-5:][::-1]

# 상위 5개의 문서 출력
print("상위 5개의 문서:")
for i in top_5_indices:
    print(f"문서 내용: {retrieved_chunks[i][1]}")
    print(f"유사도 점수: {scores[i]}")
    print("-" * 50)

상위 5개의 문서:
문서 내용: 떡갈나무는 대체로 중간 크기의 나무로 알려져 있습니다. 그러나 학생들은 떡갈나무의 잎이 얼마나 큰지 궁금해합니다. 잎의 크기를 측정하는 가장 좋은 방법은 자 사용하기입니다. 자를 이용하여 잎의 길이와 너비를 정확하게 측정할 수 있습니다. 이렇게 얻은 데이터를 통해 학생들은 떡갈나무의 잎이 얼마나 큰지 알 수 있을 것입니다. 떡갈나무의 잎은 다양한 크기와 모양을 가지고 있으며, 자를 사용하여 측정하는 것은 정확한 결과를 얻을 수 있는 가장 효과적인 방법입니다. 따라서 학생들은 자를 사용하여 떡갈나무의 잎의 크기를 측정할 수 있습니다.
유사도 점수: 0.7148746848106384
--------------------------------------------------
문서 내용: 프리즘에 의해 빛이 굴절되는 현상을 배우기 위해 한 학생이 실험을 준비했습니다. 이 학생은 어떤 색상의 빛이 굴절되는지 알아내기 위해 데이터를 수집하려고 합니다. 이를 위해 사용할 가능성이 가장 높은 방법은 '관찰'입니다. 학생은 프리즘을 사용하여 빛을 투과시키고, 굴절된 빛의 색상을 관찰하여 데이터를 수집할 것입니다. 이 방법은 비교적 간단하면서도 효과적인 방법으로, 학생이 빛의 굴절 현상을 직접 관찰하고 데이터를 수집할 수 있게 해줍니다. 따라서, 이 학생이 프리즘에 의해 어떤 색상의 빛이 굴절되는지 배우기 위해 사용할 가능성이 가장 높은 방법은 '관찰'입니다.
유사도 점수: 0.6956011056900024
--------------------------------------------------
문서 내용: 나무는 시간이 지나면서 보충할 수 있는 재생 가능한 천연 자원입니다. 나무는 우리가 일상적으로 많이 사용하는 자원 중 하나로, 우리의 삶에 꼭 필요한 다양한 용도로 활용됩니다. 나무는 산림에서 자라며, 적절한 관리와 보호를 통해 지속적으로 생산될 수 있습니다. 이러한 특성으로 인해 나무는 지속 가능한 자원으로 간주되며, 환경 보호와 자원 보

In [28]:
import pandas as pd
import json

# jsonl 파일을 읽어서 DataFrame으로 변환하는 함수
def jsonl_to_dataframe(file_path):
    # 파일을 읽어 JSON 데이터를 리스트로 변환
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    
    # JSON 데이터를 'msg' 필드로 펼쳐서 DataFrame으로 변환
    df = pd.json_normalize(data, 'msg', ['eval_id'])
    
    return df

# 파일 경로를 설정하고 함수 호출
file_path = '/ir/data/eval.jsonl'  # 'eval.jsonl' 파일 경로로 변경
df = jsonl_to_dataframe(file_path)

# DataFrame 출력
print(df)

          role                                      content eval_id
0         user                    나무의 분류에 대해 조사해 보기 위한 방법은?      78
1         user                   각 나라에서의 공교육 지출 현황에 대해 알려줘.     213
2         user                          기억 상실증 걸리면 너무 무섭겠다.     107
3    assistant                                      네 맞습니다.     107
4         user                         어떤 원인 때문에 발생하는지 궁금해.     107
..         ...                                          ...     ...
255       user                                     정상파가 뭐야?     244
256       user        공에 힘이 주어졌을 때 공이 어떻게 움직이는지 과학적으로 설명해줘.     223
257       user  동물들이 피부나 털의 색깔을 통해 생존의 확률을 높이는 방식에 대해 설명해줘.     303
258       user                       메탄과 산소의 화학 반응에 대해 알려줘.     286
259       user                                복숭아 키우는 노하우좀?     266

[260 rows x 3 columns]


'Dmitri Ivanovsky가 누구야?'

In [37]:
i = 

빗방울이 점점 커지게 되는 요인은?  = 4
오토마톤의 특징에 대해 알려줘. = 7


{"docid": "84dba7b6-8ee6-4a23-b81c-8334f0d1bdc5", "src": "ko_ai2_arc__ARC_Challenge__test", "content": "해당 비료 브랜드는 식물의 빠른 성장을 위해 필요한 모든 화학 물질을 포함하고 있다고 주장합니다. 그러나, 비료에는 탄수화물을 제외한 모든 성분이 포함되어 있다고 추론할 수 있습니다. 탄수화물은 식물의 에너지원으로 중요한 역할을 합니다. 그러므로, 이 비료는 식물의 성장을 촉진하기 위한 최적의 조건을 제공할 것으로 기대할 수 있습니다."}


건설 현장에서 망치로 벽을 치는 이유는?
검색된 Chunk ID: 41ca41ac-66e3-4a6b-a604-87bf8b3a8d4d_chunk_2762
연관된 문서 ID: 41ca41ac-66e3-4a6b-a604-87bf8b3a8d4d
Chunk 내용: 한 건설 작업자가 해머로 단단한 철벽을 치려 했지만 벽은 움직이지 않았습니다. 작업자가 사용한 에너지의 일부는 열로 변환되었습니다. 철벽은 고강도 강재로 만들어져 있어서 해머로 강력하게 치더라도 움직이지 않는 것이 정상입니다. 철벽은 건물의 구조를 지탱하고 보호하기 위해 설계되었으며, 그 내부에는 철과 다른 금속들이 견고하게 결합되어 있습니다. 따라서 해머로 치는 작업은 벽을 움직이게 하는 것보다는 에너지를 전달하여 열로 변환시키는 역할을 합니다. 이러한 열 변환은 작업자가 사용한 에너지를 벽에 흡수시키고, 벽의 온도를 조절하는 역할을 합니다. 따라서 작업자가 해머로 철벽을 치는 것은 벽을 움직이게 하는 것보다는 벽과 상호작용하여 열을 발생시키는 과정입니다.

In [79]:
i

51

In [6]:
question = df.iloc[i]['content']

print(question)
query = question
query_result = client.embeddings.create(
    model="solar-embedding-1-large-passage",
    input=query
).data[0].embedding

query_embedding = np.array(query_result).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

# KNN에서 가장 유사한 5개의 chunk 검색
k = 50
distances, indices = index.search(normalized_query, k)

# 검색된 chunk 정보 확인 및 원본 문서 내용과 연결
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {chunk_info['doc_id']}")
    print(f"{chunk_info['content']}")
    print("-" * 50)

i +=1

NameError: name 'df' is not defined